In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
rs=24 # np.random.RandomState(42)
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer,KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LogisticRegressionCV, RidgeClassifierCV
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score

In [2]:
df = pd.read_csv("data01.csv")

In [3]:
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1177 entries, 0 to 1176
Columns: 51 entries, group to EF
dtypes: float64(37), int64(14)
memory usage: 469.1 KB


In [4]:
df = df.drop(columns="ID")
df = df.dropna(subset=["outcome"])
df["outcome"] = df["outcome"].astype("int")
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1176 entries, 0 to 1176
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   group                     1176 non-null   int64  
 1   outcome                   1176 non-null   int32  
 2   age                       1176 non-null   int64  
 3   gendera                   1176 non-null   int64  
 4   BMI                       962 non-null    float64
 5   hypertensive              1176 non-null   int64  
 6   atrialfibrillation        1176 non-null   int64  
 7   CHD with no MI            1176 non-null   int64  
 8   diabetes                  1176 non-null   int64  
 9   deficiencyanemias         1176 non-null   int64  
 10  depression                1176 non-null   int64  
 11  Hyperlipemia              1176 non-null   int64  
 12  Renal failure             1176 non-null   int64  
 13  COPD                      1176 non-null   int64  
 14  heart rate   

In [5]:
y = df["outcome"]
x = df.drop(columns="outcome")
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=rs,shuffle=True)
print(f"mortality ratio in train set: {y_train.sum()/len(y_train):.3f}")
print(f"mortality ratio in test set: {y_test.sum()/len(y_test):.3f}")

mortality ratio in train set: 0.132
mortality ratio in test set: 0.148


In [6]:
# TODO: visualize the data distribution using PCA or AutoEncoder

In [7]:
# resample the train dataset by simply oversampling the mortality data 
# TODO: use more complex approaches to augment the data e.g. LinearInterpolation, SMOTE, CTGAN 
yx_train = pd.concat([y_train,x_train],axis=1)
yx_train_pos = yx_train[yx_train["outcome"]==1]
yx_train_neg = yx_train[yx_train["outcome"]==0]
print(yx_train_pos.shape,yx_train_neg.shape)

(124, 50) (816, 50)


In [8]:
rs_y_train_set=[]
rs_x_train_set=[]
for n in range(1,10):
    oversampled_yx_train_pos = pd.concat([yx_train_pos for i in range(n)])
    resampled_yx_train = pd.concat([oversampled_yx_train_pos,yx_train_neg])
    rs_y_train = resampled_yx_train["outcome"]
    rs_x_train = resampled_yx_train.drop(columns="outcome")
    rs_y_train_set.append(rs_y_train)
    rs_x_train_set.append(rs_x_train)
#print(f"mortality ratio in resampled train set: {rs_y_train.sum()/len(rs_y_train):.3f}")

In [9]:
# TODO: try more preprocessing strategies and combinations
imputer1 = SimpleImputer(strategy='median')
imputer2 = IterativeImputer(max_iter=10, random_state=rs)
imputer3 = KNNImputer(n_neighbors=2, weights="uniform")

scaler1 = StandardScaler()
scaler2 = MinMaxScaler(feature_range=(0,1))

preprocess_pipe = make_pipeline(imputer1,scaler2)

In [10]:
x_train_sample = rs_x_train_set[4] # ???

In [11]:
preprocess_pipe.fit(x_train_sample)
rs_x_train_set = [preprocess_pipe.transform(x_sample) for x_sample in rs_x_train_set]
x_test = preprocess_pipe.transform(x_test)

In [12]:
def evaluate(model,x=x_test,y=y_test):
    y_pred = model.predict(x)
    print(classification_report(y,y_pred))

def evaluate_label_ratio(model,x=x_test,y=y_test):
    f1=[]
    rec=[]
    pre=[]
    for y_train,x_train in zip(rs_y_train_set,rs_x_train_set):
        model.fit(x_train,y_train)
        y_pred = model.predict(x)
        f1.append(f"{f1_score(y,y_pred):.3f}")
        rec.append(f"{recall_score(y,y_pred):.3f}")
        pre.append(f"{precision_score(y,y_pred):.3f}")
    print("f1",f1,max(f1,key=float))
    print("rec",rec)
    print("pre",pre)

In [13]:
# level 1 models (they work independently to each other)
# TODO: optimize the models modifying hyperparams / using customized training data
# TODO: try other models

models={
       "LR1":LogisticRegression(penalty="l1",class_weight=None,solver="saga",random_state=rs),
       "LR2":LogisticRegression(penalty="l2",class_weight=None,solver="saga",random_state=rs),
       "LR3":LogisticRegression(penalty="elasticnet",l1_ratio=0.1,class_weight=None,solver="saga",random_state=rs),
       "LR4":LogisticRegression(penalty="elasticnet",l1_ratio=0.5,class_weight=None,solver="saga",random_state=rs),
       "LR5":LogisticRegression(penalty="l1",class_weight={1:1,0:1},solver="saga",random_state=rs),
       "LR6":LogisticRegression(penalty="l2",class_weight={1:1,0:1},solver="saga",random_state=rs),
       "LR7":LogisticRegression(penalty="elasticnet",l1_ratio=0.1,class_weight={1:1,0:1},solver="saga",random_state=rs),
       "LR8":LogisticRegression(penalty="elasticnet",l1_ratio=0.5,class_weight={1:1,0:1},solver="saga",random_state=rs),
       "RC1":RidgeClassifier(alpha=0.0002,random_state=rs),
       "RC2":RidgeClassifier(alpha=0.0008,random_state=rs),
       "RC3":RidgeClassifier(alpha=0.0014,random_state=rs),
       "KNC1":KNeighborsClassifier(n_neighbors=3,weights="uniform"),
       "KNC2":KNeighborsClassifier(n_neighbors=5,weights="uniform"),
       "KNC3":KNeighborsClassifier(n_neighbors=7,weights="uniform"),
       "MLPC1":MLPClassifier([25],activation="logistic",solver="adam",alpha=0.0001,max_iter=200,random_state=rs),
       "MLPC2":MLPClassifier([32],activation="logistic",solver="adam",alpha=0.0003,max_iter=200,random_state=rs),
       "MLPC3":MLPClassifier([25],activation="relu",solver="adam",alpha=0.0001,max_iter=200,random_state=rs),
       "MLPC4":MLPClassifier([32],activation="relu",solver="adam",alpha=0.0003,max_iter=200,random_state=rs),
       "MLPC5":MLPClassifier([25],activation="tanh",solver="adam",alpha=0.0001,max_iter=200,random_state=rs),
       "MLPC6":MLPClassifier([32],activation="tanh",solver="adam",alpha=0.0003,max_iter=200,random_state=rs),
       "MLPC7":MLPClassifier([20,12],activation="logistic",solver="adam",alpha=0.0001,max_iter=240,random_state=rs),
       "MLPC8":MLPClassifier([25,15],activation="logistic",solver="adam",alpha=0.0003,max_iter=240,random_state=rs),
       "MLPC9":MLPClassifier([20,12],activation="tanh",solver="adam",alpha=0.0001,max_iter=240,random_state=rs),
       "MLPC10":MLPClassifier([25,15],activation="tanh",solver="adam",alpha=0.0003,max_iter=240,random_state=rs),
       "SVC1":SVC(C=1.0,kernel="rbf",random_state=rs),
       "SVC2":SVC(C=1.0,kernel="linear",random_state=rs),
       "SVC3":SVC(C=1.0,kernel="poly",degree=3,random_state=rs),
       "SVC4":SVC(C=1.0,kernel="sigmoid",random_state=rs,),
       "SVC5":SVC(C=2.0,kernel="rbf",random_state=rs),
       "SVC6":SVC(C=2.0,kernel="linear",random_state=rs),
       "SVC7":SVC(C=2.0,kernel="poly",degree=3,random_state=rs),
       "SVC8":SVC(C=2.0,kernel="sigmoid",random_state=rs,),
       "SVC9":SVC(C=1.0,kernel="poly",degree=4,random_state=rs),
       "DTC1":DecisionTreeClassifier(criterion="gini",max_depth=12,min_samples_split=2,random_state=rs),
       "DTC2":DecisionTreeClassifier(criterion="entropy",max_depth=12,min_samples_split=2,random_state=rs),
       "DTC3":DecisionTreeClassifier(criterion="gini",max_depth=28,min_samples_split=2,random_state=rs),
       "DTC4":DecisionTreeClassifier(criterion="entropy",max_depth=28,min_samples_split=2,random_state=rs),
       "ABC":AdaBoostClassifier(estimator=None,random_state=rs),
       "BC":BaggingClassifier(estimator=None,random_state=rs),
       "RFC1":RandomForestClassifier(n_estimators=30,max_depth=12,random_state=rs),
       "RFC2":RandomForestClassifier(n_estimators=60,max_depth=12,random_state=rs),
       "RFC3":RandomForestClassifier(n_estimators=90,max_depth=12,random_state=rs),
       "RFC4":RandomForestClassifier(n_estimators=30,max_depth=24,random_state=rs),
       "RFC5":RandomForestClassifier(n_estimators=60,max_depth=24,random_state=rs),
       "RFC6":RandomForestClassifier(n_estimators=90,max_depth=24,random_state=rs),
       "HGBC":HistGradientBoostingClassifier(random_state=rs)
       }

In [14]:
# TODO: adopt level 2 model to build ensemble models e.g. Stacking

In [15]:
# TODO: visualizing model contribution / importances
# TODO(?): drop some unimportant / bad-behaving models, and train the ensemble model again

In [16]:
# TODO: visualizing feature importances
# TODO(?): drop some unimportant features according to the visualization, and train the models again

In [17]:
for name,model in models.items():
    print(name)
    evaluate_label_ratio(model,x_test,y_test)
    print("")

LR1
f1 ['0.400', '0.469', '0.550', '0.479', '0.451', '0.467', '0.448', '0.430', '0.427'] 0.550
rec ['0.286', '0.429', '0.629', '0.657', '0.657', '0.714', '0.743', '0.743', '0.800']
pre ['0.667', '0.517', '0.489', '0.377', '0.343', '0.347', '0.321', '0.302', '0.292']

LR2
f1 ['0.440', '0.508', '0.545', '0.500', '0.447', '0.449', '0.444', '0.423', '0.446'] 0.545
rec ['0.314', '0.457', '0.600', '0.657', '0.657', '0.686', '0.743', '0.743', '0.829']
pre ['0.733', '0.571', '0.500', '0.404', '0.338', '0.333', '0.317', '0.295', '0.305']

LR3
f1 ['0.408', '0.516', '0.545', '0.500', '0.447', '0.449', '0.448', '0.423', '0.446'] 0.545
rec ['0.286', '0.457', '0.600', '0.657', '0.657', '0.686', '0.743', '0.743', '0.829']
pre ['0.714', '0.593', '0.500', '0.404', '0.338', '0.333', '0.321', '0.295', '0.305']

LR4
f1 ['0.408', '0.476', '0.557', '0.500', '0.447', '0.463', '0.441', '0.430', '0.424'] 0.557
rec ['0.286', '0.429', '0.629', '0.657', '0.657', '0.714', '0.743', '0.743', '0.800']
pre ['0.714', '

In [18]:
# level 1 models (they work independently to each other)
# TODO: optimize the models modifying hyperparams / using customized training data
# TODO: try other models
rs=24
models2={
       "LR1":LogisticRegression(penalty="l1",class_weight=None,solver="saga",random_state=rs),
       "LR2":LogisticRegression(penalty="l2",class_weight=None,solver="saga",random_state=rs),
       "LR3":LogisticRegression(penalty="elasticnet",l1_ratio=0.1,class_weight=None,solver="saga",random_state=rs),
       "LR4":LogisticRegression(penalty="elasticnet",l1_ratio=1,class_weight=None,solver="saga",random_state=rs),
       "LR5":LogisticRegression(penalty="l1",class_weight={1:1.8,0:1},solver="saga",random_state=rs),
       "LR6":LogisticRegression(penalty="l2",class_weight={1:1.8,0:1},solver="saga",random_state=rs),
       "LR7":LogisticRegression(penalty="elasticnet",l1_ratio=0.1,class_weight={1:1.8,0:1},solver="saga",random_state=rs),
       "LR8":LogisticRegression(penalty="elasticnet",l1_ratio=1,class_weight={1:1.8,0:1},solver="saga",random_state=rs),
       "RC1":RidgeClassifier(alpha=0.75,random_state=rs),
       "RC2":RidgeClassifier(alpha=1,random_state=rs),
       "RC3":RidgeClassifier(alpha=1.5,random_state=rs),
       #"KNC1":KNeighborsClassifier(n_neighbors=11,weights="uniform"),
       #"KNC2":KNeighborsClassifier(n_neighbors=17,weights="uniform"),
       #"KNC3":KNeighborsClassifier(n_neighbors=23,weights="uniform"),
       #"MLPC1":MLPClassifier([25],activation="logistic",solver="adam",alpha=0.0001,max_iter=200,random_state=rs),
       #"MLPC2":MLPClassifier([32],activation="logistic",solver="adam",alpha=0.0003,max_iter=200,random_state=rs),
       #"MLPC3":MLPClassifier([25],activation="relu",solver="adam",alpha=0.0001,max_iter=200,random_state=rs),
       #"MLPC4":MLPClassifier([32],activation="relu",solver="adam",alpha=0.0003,max_iter=200,random_state=rs),
       #"MLPC5":MLPClassifier([25],activation="tanh",solver="adam",alpha=0.0001,max_iter=200,random_state=rs),
       #"MLPC6":MLPClassifier([32],activation="tanh",solver="adam",alpha=0.0003,max_iter=200,random_state=rs),
       "MLPC7":MLPClassifier([20,12],activation="logistic",solver="adam",alpha=0.0001,max_iter=300,random_state=rs),
       "MLPC8":MLPClassifier([25,15],activation="logistic",solver="adam",alpha=0.0003,max_iter=300,random_state=rs),
       "MLPC9":MLPClassifier([20,12],activation="tanh",solver="adam",alpha=0.0001,max_iter=300,random_state=rs),
       "MLPC10":MLPClassifier([25,15],activation="tanh",solver="adam",alpha=0.0003,max_iter=300,random_state=rs),
       "MLPC11":MLPClassifier([25,25,12],activation="logistic",solver="adam",alpha=0.0003,max_iter=360,random_state=rs),
       "MLPC12":MLPClassifier([32,32,15],activation="logistic",solver="adam",alpha=0.0005,max_iter=360,random_state=rs),
       "MLPC13":MLPClassifier([25,25,12],activation="relu",solver="adam",alpha=0.0003,max_iter=360,random_state=rs),
       "MLPC14":MLPClassifier([32,32,15],activation="relu",solver="adam",alpha=0.0005,max_iter=360,random_state=rs),
       "MLPC15":MLPClassifier([25,25,12],activation="tanh",solver="adam",alpha=0.0003,max_iter=360,random_state=rs),
       "MLPC16":MLPClassifier([32,32,15],activation="tanh",solver="adam",alpha=0.0005,max_iter=360,random_state=rs),
       "SVC1":SVC(C=2.0,kernel="rbf",random_state=rs),
       "SVC2":SVC(C=2.0,kernel="linear",random_state=rs),
       "SVC3":SVC(C=2.0,kernel="poly",degree=3,random_state=rs),
       "SVC4":SVC(C=2.0,kernel="sigmoid",random_state=rs,),
       "SVC5":SVC(C=0.5,kernel="rbf",random_state=rs),
       "SVC6":SVC(C=0.5,kernel="linear",random_state=rs),
       "SVC7":SVC(C=0.5,kernel="poly",degree=3,random_state=rs),
       "SVC8":SVC(C=0.5,kernel="sigmoid",random_state=rs,),
       "SVC9":SVC(C=2.0,kernel="poly",degree=5,random_state=rs),
       "SVC10":SVC(C=0.5,kernel="poly",degree=5,random_state=rs),
       "DTC1":DecisionTreeClassifier(criterion="gini",max_depth=18,min_samples_split=2,random_state=rs),
       "DTC2":DecisionTreeClassifier(criterion="entropy",max_depth=18,min_samples_split=2,random_state=rs),
       "DTC3":DecisionTreeClassifier(criterion="gini",max_depth=34,min_samples_split=2,random_state=rs),
       "DTC4":DecisionTreeClassifier(criterion="entropy",max_depth=34,min_samples_split=2,random_state=rs),
       "ABC":AdaBoostClassifier(estimator=None,random_state=rs),
       "ABC2":AdaBoostClassifier(estimator=LogisticRegression(random_state=rs),random_state=rs),
       "BC":BaggingClassifier(estimator=None,random_state=rs),
       "BC2":BaggingClassifier(estimator=LogisticRegression(random_state=rs),random_state=rs),
       "RFC1":RandomForestClassifier(n_estimators=3,max_depth=16,random_state=rs),
       "RFC2":RandomForestClassifier(n_estimators=6,max_depth=16,random_state=rs),
       "RFC3":RandomForestClassifier(n_estimators=9,max_depth=16,random_state=rs),
       "RFC4":RandomForestClassifier(n_estimators=3,max_depth=24,random_state=rs),
       "RFC5":RandomForestClassifier(n_estimators=6,max_depth=24,random_state=rs),
       "RFC6":RandomForestClassifier(n_estimators=9,max_depth=24,random_state=rs),
       "HGBC":HistGradientBoostingClassifier(random_state=rs)
       }

In [19]:
for name,model in models2.items():
    print(name)
    evaluate_label_ratio(model,x_test,y_test)
    print("")

LR1
f1 ['0.400', '0.469', '0.550', '0.479', '0.451', '0.467', '0.448', '0.430', '0.427'] 0.550
rec ['0.286', '0.429', '0.629', '0.657', '0.657', '0.714', '0.743', '0.743', '0.800']
pre ['0.667', '0.517', '0.489', '0.377', '0.343', '0.347', '0.321', '0.302', '0.292']

LR2
f1 ['0.440', '0.508', '0.545', '0.500', '0.447', '0.449', '0.444', '0.423', '0.446'] 0.545
rec ['0.314', '0.457', '0.600', '0.657', '0.657', '0.686', '0.743', '0.743', '0.829']
pre ['0.733', '0.571', '0.500', '0.404', '0.338', '0.333', '0.317', '0.295', '0.305']

LR3
f1 ['0.408', '0.516', '0.545', '0.500', '0.447', '0.449', '0.448', '0.423', '0.446'] 0.545
rec ['0.286', '0.457', '0.600', '0.657', '0.657', '0.686', '0.743', '0.743', '0.829']
pre ['0.714', '0.593', '0.500', '0.404', '0.338', '0.333', '0.321', '0.295', '0.305']

LR4
f1 ['0.400', '0.469', '0.550', '0.479', '0.451', '0.467', '0.448', '0.430', '0.427'] 0.550
rec ['0.286', '0.429', '0.629', '0.657', '0.657', '0.714', '0.743', '0.743', '0.800']
pre ['0.667', '

In [20]:
# level 1 models (they work independently to each other)
# TODO: optimize the models modifying hyperparams / using customized training data
# TODO: try other models
rs=123
models3={
       "LR1":LogisticRegressionCV(penalty="l1",class_weight=None,solver="saga",random_state=rs),
       "LR2":LogisticRegressionCV(penalty="l2",class_weight=None,solver="saga",random_state=rs),    
       "LR5":LogisticRegressionCV(penalty="l1",class_weight={1:1.8,0:1},solver="saga",random_state=rs),
       "LR6":LogisticRegressionCV(penalty="l2",class_weight={1:1.8,0:1},solver="saga",random_state=rs),
       "RC1":RidgeClassifierCV(alphas=(0.1, 1.0, 10.0)),
       "RC2":RidgeClassifierCV(alphas=(0.01, 0.1, 1)),
       "RC3":RidgeClassifierCV(alphas=(0.001, 0.01, 0.1)),
       "MLPC11":MLPClassifier([25,25,12],activation="logistic",solver="adam",alpha=0.0001,max_iter=440,random_state=rs),
       "MLPC12":MLPClassifier([32,32,15],activation="logistic",solver="adam",alpha=0.0001,max_iter=440,random_state=rs),
       "MLPC13":MLPClassifier([25,25,12],activation="relu",solver="adam",alpha=0.0001,max_iter=440,random_state=rs),
       "MLPC14":MLPClassifier([32,32,15],activation="relu",solver="adam",alpha=0.0001,max_iter=440,random_state=rs),
       "MLPC15":MLPClassifier([25,25,12],activation="tanh",solver="adam",alpha=0.0001,max_iter=440,random_state=rs),
       "MLPC16":MLPClassifier([32,32,15],activation="tanh",solver="adam",alpha=0.0001,max_iter=440,random_state=rs),

       
       }

In [21]:
for name,model in models3.items():
    print(name)
    evaluate_label_ratio(model,x_test,y_test)
    print("")

LR1
f1 ['0.519', '0.507', '0.500', '0.458', '0.462', '0.444', '0.429', '0.450', '0.438'] 0.519
rec ['0.400', '0.514', '0.600', '0.629', '0.686', '0.686', '0.686', '0.771', '0.800']
pre ['0.737', '0.500', '0.429', '0.361', '0.348', '0.329', '0.312', '0.318', '0.301']

LR2
f1 ['0.462', '0.507', '0.537', '0.458', '0.462', '0.444', '0.442', '0.450', '0.438'] 0.537
rec ['0.343', '0.514', '0.629', '0.629', '0.686', '0.686', '0.714', '0.771', '0.800']
pre ['0.706', '0.500', '0.468', '0.361', '0.348', '0.329', '0.321', '0.318', '0.301']

LR5
f1 ['0.473', '0.478', '0.438', '0.448', '0.438', '0.430', '0.419', '0.408', '0.405'] 0.478
rec ['0.371', '0.629', '0.657', '0.743', '0.800', '0.829', '0.886', '0.886', '0.914']
pre ['0.650', '0.386', '0.329', '0.321', '0.301', '0.290', '0.274', '0.265', '0.260']

LR6
f1 ['0.481', '0.421', '0.466', '0.452', '0.444', '0.430', '0.419', '0.405', '0.392'] 0.481
rec ['0.371', '0.343', '0.686', '0.743', '0.800', '0.829', '0.886', '0.886', '0.886']
pre ['0.684', '

In [22]:
# level 1 models (they work independently to each other)
# TODO: optimize the models modifying hyperparams / using customized training data
# TODO: try other models
rs=123
models4={
       "ABC1":AdaBoostClassifier(estimator=LogisticRegression(random_state=rs),n_estimators=20,random_state=rs),
       "ABC2":AdaBoostClassifier(estimator=LogisticRegression(random_state=rs),n_estimators=50,random_state=rs),
       "ABC3":AdaBoostClassifier(estimator=LogisticRegression(random_state=rs),n_estimators=100,random_state=rs),
       "BC1":BaggingClassifier(estimator=LogisticRegression(random_state=rs),n_estimators=5,random_state=rs),
       "BC2":BaggingClassifier(estimator=LogisticRegression(random_state=rs),n_estimators=10,random_state=rs),
       "BC3":BaggingClassifier(estimator=LogisticRegression(random_state=rs),n_estimators=20,random_state=rs),
       "BC4":BaggingClassifier(estimator=SVC(kernel="poly",degree=3,random_state=rs),n_estimators=5,random_state=rs),
       "BC5":BaggingClassifier(estimator=SVC(kernel="poly",degree=3,random_state=rs),n_estimators=10,random_state=rs),
       "BC6":BaggingClassifier(estimator=SVC(kernel="poly",degree=3,random_state=rs),n_estimators=20,random_state=rs),

       
       }

In [23]:
for name,model in models4.items():
    print(name)
    evaluate_label_ratio(model,x_test,y_test)
    print("")

ABC1
f1 ['0.000', '0.158', '0.318', '0.393', '0.395', '0.455', '0.471', '0.440', '0.400'] 0.471
rec ['0.000', '0.086', '0.200', '0.314', '0.486', '0.714', '0.914', '0.943', '0.943']
pre ['0.000', '1.000', '0.778', '0.524', '0.333', '0.333', '0.317', '0.287', '0.254']

ABC2
f1 ['0.200', '0.340', '0.400', '0.513', '0.460', '0.509', '0.465', '0.463', '0.435'] 0.513
rec ['0.114', '0.229', '0.314', '0.571', '0.657', '0.829', '0.857', '0.886', '0.914']
pre ['0.800', '0.667', '0.550', '0.465', '0.354', '0.367', '0.319', '0.313', '0.286']

ABC3
f1 ['0.356', '0.453', '0.514', '0.494', '0.466', '0.487', '0.475', '0.465', '0.449'] 0.514
rec ['0.229', '0.343', '0.543', '0.600', '0.686', '0.800', '0.829', '0.857', '0.886']
pre ['0.800', '0.667', '0.487', '0.420', '0.353', '0.350', '0.333', '0.319', '0.301']

BC1
f1 ['0.383', '0.492', '0.493', '0.500', '0.455', '0.432', '0.430', '0.439', '0.438'] 0.500
rec ['0.257', '0.429', '0.514', '0.657', '0.657', '0.686', '0.743', '0.771', '0.800']
pre ['0.750'